In [1]:
from __future__ import division, print_function
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from graph import ECG_model
from ulits import *
from config import get_config
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
from tensorflow.keras import layers

In [ ]:
def train(config, X, y, Xval=None, yval=None):
    classes = ['N', 'V', '/', 'A', 'F', '~']
    Xe = np.expand_dims(X, axis=2)
    if not config.split:
        from sklearn.model_selection import train_test_split
        Xe, Xval, y, yval = train_test_split(Xe, y, test_size=0.2, random_state=1)
        
    else:
        if Xval is not None and yval is not None:  
            Xval = np.expand_dims(Xval, axis=2)  
            (m, n) = y.shape
            y = y.reshape((m, 1, n))
            (mvl, nvl) = yval.shape
            yval = yval.reshape((mvl, 1, nvl))
        else:
            print("Xval or yval is None. Unable to train with validation data.")
            return  

    model = ECG_model(config)

    # Define directory and create it if necessary
    model_save_dir = 'models'
    mkdir_recursive(model_save_dir)

    # Convert to TensorFlow Lite model
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    tflite_model_save_path = os.path.join(model_save_dir, f'{config.feature}--latest.tflite')
    with open(tflite_model_save_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Model saved as TensorFlow Lite model at {tflite_model_save_path}")

    # Define model checkpoint path using your custom directory creation
    model_checkpoint_path = os.path.join(model_save_dir, 'checkpoint-{epoch:02d}-{val_loss:.2f}.h5')

    # Setup callbacks
    callbacks = [
        EarlyStopping(patience=config.patience, verbose=1),
        ReduceLROnPlateau(factor=0.5, patience=3, min_lr=0.01, verbose=1),
        ModelCheckpoint(model_checkpoint_path, monitor='val_loss', save_best_only=False, verbose=1)  # Removed period=10 for testing
    ]
    
    model.fit(Xe, y, validation_data=(Xval, yval), epochs=config.epochs,
              batch_size=config.batch, callbacks=callbacks)
    print_results(config, model, Xval, yval, classes)

def main(config):
    print('feature:', config.feature)
    
    if config.split:  # Simplified condition
        (X, y, Xval, yval) = loaddata(config.input_size, config.feature)
        train(config, X, y, Xval, yval)
    else:
        (X, y) = loaddata_nosplit(config.input_size, config.feature)
        train(config, X, y)
        
if __name__ == "__main__":
    config = get_config()
    main(config)

feature: MLII
Shape of X: (36902, 256)
Shape of y: (36902, 6)
Shape of Xval: (13479, 256)
Shape of yval: (13479, 6)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)            │ (None, 256, 1)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 256, 32)           │             544 │ input[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 256, 32)           │             128 │ conv1d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 256, 32)           │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_1 (Conv1D)             │ (None, 128, 32)           │           1,056 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 128, 32)           │             128 │ conv1d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 128, 32)           │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128, 32)           │               0 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_2 (Conv1D)             │ (None, 128, 32)           │          16,416 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_3 (Conv1D)             │ (None, 128, 32)           │           1,056 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 128, 32)           │               0 │ conv1d_2[0][0],            │
│                               │                           │                 │ conv1d_3[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 128, 32)           │             128 │ add[0][0]                  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 128, 32)           │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_4 (Conv1D)             │ (None, 128, 32)           │          16,416 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 128, 32)           │             128 │ conv1d_4[0][0]             │
│ (BatchNormalization)          │                           │               

 Total params: 382,758 (1.46 MB)

 Trainable params: 380,774 (1.45 MB)

 Non-trainable params: 1,984 (7.75 KB)

INFO:tensorflow:Assets written to: C:\Users\majdt\AppData\Local\Temp\tmp0ye1eagb\assets


INFO:tensorflow:Assets written to: C:\Users\majdt\AppData\Local\Temp\tmp0ye1eagb\assets
